In [478]:
# Import libraries
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
import csv
import pandas as pd

In [479]:
# Setup project target, example: Data Science Junior Jobs in Florianopolis
url = 'https://www.linkedin.com/jobs/search?keywords=junior%20data%20analyst&location=Florianopolis&refresh=true'

In [480]:
# Setup chromedriver 
try:
    wd = webdriver.Chrome()
except:
    wd = webdriver.Chrome(ChromeDriverManager().install())
wd.get(url)




====== WebDriver manager ======
Current google-chrome version is 105.0.5195
Get LATEST chromedriver version for 105.0.5195 google-chrome
Driver [/home/fred/.wdm/drivers/chromedriver/linux64/105.0.5195.52/chromedriver] found in cache
/tmp/ipykernel_45406/1428814053.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())


In [481]:
from selenium.webdriver.common.by import By

In [482]:
# Get the number of jobs available
no_of_jobs = int(wd.find_element(By.CSS_SELECTOR, value='h1>span').get_attribute('innerText'))
print(no_of_jobs)

140


In [483]:
# Browse all the jobs
i = 2
while i <= int(no_of_jobs/25)+1: 
    wd.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    i = i + 1
    try:
        wd.find_element(By.XPATH, value='/html/body/main/div/section/button').click()
        time.sleep(5)
    except:
        pass
        time.sleep(5)

    

In [484]:
# Find all the jobs
job_lists = wd.find_element(By.CLASS_NAME, value='jobs-search__results-list')
jobs = job_lists.find_elements(By.TAG_NAME, value='li') # return a list

In [485]:
# Check the number of jobs on the list
len(jobs)

124

In [486]:
# Load job details into dataframe
job_id = []
job_title = []
company_name = []
location = []
date = []
job_link = []
jd = []
seniority = []
emp_type = []
job_func = []
industries = []

for job in jobs:
        job_id0 = job.find_element(By.CLASS_NAME, value='base-card').get_attribute('data-entity-urn')
        job_id.append(job_id0)
 
        job_title0 = job.find_element(By.CSS_SELECTOR, value='h3').text
        job_title.append(job_title0)
 
        company_name0 = job.find_element(By.CSS_SELECTOR, value='h4').text
        company_name.append(company_name0)
 
        location0 = job.find_element(By.CLASS_NAME, value='job-search-card__location').text
        location.append(location0)
 
        date0 = job.find_element(By.CSS_SELECTOR, value='div>div>time').get_attribute('datetime')
        date.append(date0)
 
        job_link0 = job.find_element(By.CSS_SELECTOR, value='a').get_attribute('href')
        job_link.append(job_link0) 


        # Creating the soon to be filled columns of each job
        
        jd.append(" ")
        seniority.append(" ")
        emp_type.append(" ")
        job_func.append(" ")
        industries.append(" ")



In [487]:
print(len(jobs))

124


In [488]:
# Load data into DataFrame for further analysis
job_data = pd.DataFrame({'ID': job_id,
                         'Date': date,
                         'Company': company_name,
                         'Title': job_title,
                         'Location': location,
                         'Link': job_link,
                         'Description': jd,
                         'Level': seniority,
                         'Type': emp_type,
                         'Function': job_func,
                         'Industry': industries
                         })

# cleaning description column
#job_data['Description'] = job_data['Description'].str.replace('\n', ' ')
job_data.to_csv('linkedin_jobs_data-3.csv', index=False)

In [489]:
# Load further details

filename = 'linkedin_jobs_data-3.csv'

# Opening csv file to read job links

with open(filename, 'r') as csvfile: 
    csvreader = csv.reader(csvfile)
    next(csvreader)

    for job in csvreader:
        # indicates which job is going to be updated
        CSVtabelIndex = 0

        url = job[5]
        print("Currently scraped url:",url)
        wd.get(url)

        # selecting elements that hold the job description (about)
        jd0 = wd.find_element(By.CLASS_NAME, value='show-more-less-html__markup').text
        print(jd0)
        
        # selecting elements that hold job 4 criterias 
        job = wd.find_elements(By.CLASS_NAME, value='description__job-criteria-item')

        seniority0 = job[0].find_element(By.CLASS_NAME, value='description__job-criteria-text').text
        print(seniority0)
        
        emp_type0 = job[1].find_element(By.CLASS_NAME, value='description__job-criteria-text').text
        print(emp_type0)
        
        job_func0 = job[2].find_element(By.CLASS_NAME, value='description__job-criteria-text').text
        print(job_func0)

        industries0 = job[3].find_element(By.CLASS_NAME, value='description__job-criteria-text').text
        print(industries0)


        jd[CSVtabelIndex] = jd0
        seniority[CSVtabelIndex] = seniority0
        emp_type[CSVtabelIndex]= emp_type0
        job_func[CSVtabelIndex] = job_func0         
        industries[CSVtabelIndex] = industries0

        CSVtabelIndex = CSVtabelIndex + 1
        break
        time.sleep(5)


Currently scraped url: https://br.linkedin.com/jobs/view/analista-de-processos-foco-em-analise-de-dados-at-incentiv-me-3272108081?refId=lOGmTxkQVSghHlJwLMxneg%3D%3D&trackingId=a%2FVx6RPH%2BTlx2xCoo%2BXxcQ%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card
Como Analista de Processos Pleno você terá a missão de contribuir para implementação e manutenção do sistema de gestão de processos de negócios e qualidade transformando as fontes de dados em informações, facilitando a tomada de decisão dos gestores da empresa, garantindo a aderência aos parâmetros das normas e exigências do mercado, dentro das políticas definidas pela empresa.



Nesse sentido precisamos que você tenha um perfil analítico, comunicativo, bom raciocínio lógico e que tenha muita proatividade para solucionar os problemas que possam surgir durante a jornada. Se identificou.















Aqui você encontrará gente do bem, positiva, que ama o que faz e que faz acontecer!

Gente que acredita na riqueza da 

In [490]:


print(len(job_id),len(date),len(company_name),len(job_title),len(location),len(job_link),len(jd),len(seniority),len(emp_type),len(job_func),len(industries))

#Load data into DataFrame for further analysis
job_data = pd.DataFrame({'ID': job_id,
                         'Date': date,
                         'Company': company_name,
                         'Title': job_title,
                         'Location': location,
                         'Link': job_link,
                         'Description': jd,
                         'Level': seniority,
                         'Type': emp_type,
                         'Function': job_func,
                         'Industry': industries
                         })

job_data.to_csv('linkedin_jobs_data-3.csv', index=False)

124 124 124 124 124 124 124 124 124 124 124
